# Home Credit Prediction: Data Cleaning of Credit Card Table


For an up-to-date version / full view of the plotly - plots, please, go to

Data Cleaning - Credit Card: https://drive.google.com/file/d/17Xt2BNZ_AbtZDq3u5fUCz20u-27kmxfp/view?usp=sharing

List of all notebooks and resources for this project: https://drive.google.com/file/d/1Z8vPNZAcivWOxeh3UKFfeARbQCMkQ_NR/view?usp=sharing

## Import Modules

In [ ]:
%%capture
#! pip install -q pingouin
#! pip install -q scikit-optimize
! pip install -q scikit-optimize

In [ ]:
import numpy as np
import pandas as pd

import sys
import os
import warnings
from importlib import reload

from dask import dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px

from google.colab import drive
drive.mount("/content/gdrive")

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')

Mounted at /content/gdrive


In [ ]:
home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Portfolio/ML_HomeCredit_DefaultRiskEvaluation/'

### Functions

The Python-file with the functions is at
https://drive.google.com/file/d/1IsRcGuolR4Hnu6bGe44GcS_0UQPFM59h/view?usp=sharing

In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.append(home_folder)
import driskfunc as dfunc

# 1. Load and Update Data

data source: https://storage.googleapis.com/341-home-credit-default/home-credit-default-risk.zip

description: https://storage.googleapis.com/341-home-credit-default/Home%20Credit%20Default%20Risk.pdf

In [ ]:
HCdescr = pd.read_csv(home_folder+'data/HomeCredit_columns_description.csv', encoding='latin1') #, dtype=dtype)


In [ ]:
HCdescr.loc[HCdescr.Table == 'credit_card_balance.csv']

,Unnamed: 0,Table,Row,Description,Special
150,153,credit_card_balance.csv,SK_ID_PREV,"ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)",hashed
151,154,credit_card_balance.csv,SK_ID_CURR,ID of loan in our sample,hashed
152,155,credit_card_balance.csv,MONTHS_BALANCE,Month of balance relative to application date (-1 means the freshest balance date),time only relative to the application
153,156,credit_card_balance.csv,AMT_BALANCE,Balance during the month of previous credit,NaN
154,157,credit_card_balance.csv,AMT_CREDIT_LIMIT_ACTUAL,Credit card limit during the month of the previous credit,NaN
155,158,credit_card_balance.csv,AMT_DRAWINGS_ATM_CURRENT,Amount drawing at ATM during the month of the previous credit,NaN
156,159,credit_card_balance.csv,AMT_DRAWINGS_CURRENT,Amount drawing during the month of the previous credit,NaN
157,160,credit_card_balance.csv,AMT_DRAWINGS_OTHER_CURRENT,Amount of other drawings during the month of the previous credit,NaN
158,161,credit_card_balance.csv,AMT_DRAWINGS_POS_CURRENT,Amount drawing or buying goods during the month of the previous credit,NaN
159,162,credit_card_balance.csv,AMT_INST_MIN_REGULARITY,Minimal installment for this month of the previous credit,NaN


In [ ]:
csv_cc = home_folder+'data/credit_card_balance.csv'
HCapp_cc = dd.read_csv(csv_cc)

df = HCapp_cc
df_name = 'HCapp credit card balance'

In [ ]:
df.npartitions

6

In [ ]:
df = df.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=False)
df = df.sort_values(by=['SK_ID_CURR','SK_ID_PREV'], ascending=True)

In [ ]:
df.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
584804,1489396,100006,-1,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
520387,1489396,100006,-2,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
59712,1489396,100006,-3,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
112079,1489396,100006,-4,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
11293,1489396,100006,-5,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
348325,1489396,100006,-6,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
171936,1843384,100011,-2,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0
293842,1843384,100011,-3,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0
51047,1843384,100011,-4,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0
107800,1843384,100011,-5,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0


In [ ]:
size_df = [df.shape[0].compute(),  df.shape[1]]

print('The dataset', df_name, 'has', size_df[0], 'rows and', size_df[1], 'features.')

The dataset HCapp credit card balance has 3840312 rows and 23 features.


# 2. Data Cleaning

* Handling missing values.
* Removing duplicate samples and features.
* Remove unneccessary columns/rows.
* Treating (here rather checking) the outliers.

## Check Missing Values and Duplicates

Overview of amounts of Nan and of data type:

In [ ]:
dfunc.count_dtypes(df, name = df_name)


The dataset HCapp credit card balance has:
15 features of type float64.
7 features of type int64.
1 features of type string.


In [ ]:
%%time
%reload_ext autoreload

nan_overview_df = dfunc.nan_type_overview_dd(df, size_df[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

CPU times: user 25.7 s, sys: 3.11 s, total: 28.8 s
Wall time: 19.7 s


,type,NaN[abs],NaN[%]
SK_ID_PREV,int64,0,0.000000
SK_ID_CURR,int64,0,0.000000
MONTHS_BALANCE,int64,0,0.000000
AMT_BALANCE,float64,0,0.000000
AMT_CREDIT_LIMIT_ACTUAL,int64,0,0.000000
AMT_DRAWINGS_ATM_CURRENT,float64,749816,19.500000
AMT_DRAWINGS_CURRENT,float64,0,0.000000
AMT_DRAWINGS_OTHER_CURRENT,float64,749816,19.500000
AMT_DRAWINGS_POS_CURRENT,float64,749816,19.500000
AMT_INST_MIN_REGULARITY,float64,305236,7.900000


### Duplicates Check

In [ ]:
%reload_ext autoreload

df_dup = dfunc.get_dup_dd(df, name=df_name, size=size_df[0])

Total number of duplicates in " HCapp credit card balance " : 0 ( 0.0 %).


## Drops based on Statistical Overview

### Overview

In [ ]:
df.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
SK_ID_PREV,3840312.0,1904503.6,536469.5,1000018.0,1444518.0,1905903.0,2379481.0,2843496.0
SK_ID_CURR,3840312.0,278324.2,102704.5,100006.0,183226.0,262223.0,355993.0,456250.0
MONTHS_BALANCE,3840312.0,-34.5,26.7,-96.0,-55.0,-27.0,-11.0,-1.0
AMT_BALANCE,3840312.0,58300.2,106307.0,-420250.2,0.0,0.0,90887.1,1505902.2
AMT_CREDIT_LIMIT_ACTUAL,3840312.0,153808.0,165145.7,0.0,45000.0,135000.0,180000.0,1350000.0
AMT_DRAWINGS_ATM_CURRENT,3090496.0,5961.3,28225.7,-6827.3,0.0,0.0,0.0,2115000.0
AMT_DRAWINGS_CURRENT,3840312.0,7433.4,33846.1,-6211.6,0.0,0.0,0.0,2287098.3
AMT_DRAWINGS_OTHER_CURRENT,3090496.0,288.2,8202.0,0.0,0.0,0.0,0.0,1529847.0
AMT_DRAWINGS_POS_CURRENT,3090496.0,2968.8,20796.9,0.0,0.0,0.0,0.0,2239274.2
AMT_INST_MIN_REGULARITY,3535076.0,3540.2,5600.2,0.0,0.0,0.0,6750.0,202882.0


The distribution characterisitcs of AMT_BALANCE,AMT_RECEIVABLE_PRINCIPAL, AMT_RECIVABLE, AMT_TOTAL_RECEIVABLE look strikingly similar. For CNT_DRAWINGS_CURRENT and CNT_DRAWINGS_POS_CURRENT, too.

In [ ]:
df_obj = df.describe(exclude=np.number).compute().T
df_obj['freq'] = pd.to_numeric(df_obj['freq'])
df_obj['count'] = pd.to_numeric(df_obj['count'])
df_obj['freq[%]'] = (df_obj['freq']/df_obj['count']*100).astype('float')
df_obj.round(1)

,unique,count,top,freq,freq[%]
NAME_CONTRACT_STATUS,7,3840312,Active,3698436,96.3


### Other Checks and Modifications

#### INSTALMENTS paid

In [ ]:
df = df.rename(columns = {'CNT_INSTALMENT_MATURE_CUM': 'INSTALMENTS_paid'})  # align with POS CASH

#### NAME_CONTRACT_STATUS

In [ ]:
df.NAME_CONTRACT_STATUS.value_counts().compute()

,count
NAME_CONTRACT_STATUS,
Active,3698436
Completed,128918
Signed,11058
Demand,1365
Sent proposal,513
Refused,17
Approved,5


Inspection shows that a month with 'Refused' (typicall the very first month of the credit running, i.e. oldest month in payment list of a specific SK_ID_PREV) does not end the contract, but continues with 'Active' in the following months.
Even if 'Refused' gives a hint on the credit worthiness, with just 17 rows out of >3 Mio. it provides too few cases to show an effect. Maybe 'Refused' rather indicates mistakes when filling the application form.

In [ ]:
df_cstat = df['NAME_CONTRACT_STATUS']
df_cstat = df_cstat.mask((df_cstat=='Signed'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Sent proposal'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Approved'), 'Initialized')
df_cstat = df_cstat.mask((df_cstat=='Refused'), 'Initialized')
df['NAME_CONTRACT_STATUS_red'] = df_cstat
df['NAME_CONTRACT_STATUS_red'].value_counts().compute()

,count
NAME_CONTRACT_STATUS_red,
Active,3698436
Completed,128918
Initialized,11593
Demand,1365


## Aggregate by SK_ID_PREV

In [ ]:
df_stat = df.categorize(columns = ['NAME_CONTRACT_STATUS_red']).NAME_CONTRACT_STATUS_red
df_status_cat = dd.get_dummies(df_stat, prefix = 'STATUS', dtype=int)

df_with_dummies = dd.concat([df, df_status_cat], axis=1)

#df_with_dummies.head(20)

In [ ]:
df_with_dummies.head(20)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,INSTALMENTS_paid,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_red,STATUS_Active,STATUS_Completed,STATUS_Demand,STATUS_Initialized
584804,1489396,100006,-1,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0,Active,1,0,0,0
520387,1489396,100006,-2,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0,Active,1,0,0,0
59712,1489396,100006,-3,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0,Active,1,0,0,0
112079,1489396,100006,-4,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0,Active,1,0,0,0
11293,1489396,100006,-5,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0,Active,1,0,0,0
348325,1489396,100006,-6,0.0,270000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0,Active,1,0,0,0
171936,1843384,100011,-2,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0,Active,1,0,0,0
293842,1843384,100011,-3,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0,Active,1,0,0,0
51047,1843384,100011,-4,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0,Active,1,0,0,0
107800,1843384,100011,-5,0.0,90000,0.0,0.0,0.0,0.0,0.0,563.355,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,Active,0,0,Active,1,0,0,0


In [ ]:
df_mean = df_with_dummies.groupby("SK_ID_PREV").mean(numeric_only=True)
df_mean = df_mean.drop(columns='MONTHS_BALANCE')

df_count = df_with_dummies.groupby("SK_ID_PREV")['MONTHS_BALANCE'].count().to_frame()
df_count = df_count.rename(columns={'MONTHS_BALANCE': 'MONTHS_BALANCE_count'})

df_installment = df_with_dummies.groupby("SK_ID_PREV")['MONTHS_BALANCE',
                                                       'INSTALMENTS_paid',
                                                       'NAME_CONTRACT_STATUS',
                                                       'NAME_CONTRACT_STATUS_red'
                                                       ].first()

install_cols = {'MONTHS_BALANCE': 'MONTHS_BALANCE_latest',
                'INSTALMENTS_paid': 'INSTALMENTS_paid_latest',
                'NAME_CONTRACT_STATUS': 'NAME_CONTRACT_STATUS_latest',
                'NAME_CONTRACT_STATUS_red': 'NAME_CONTRACT_STATUS_red_latest'
                }

df_installment = df_installment.rename(columns=install_cols)


df_count_inst = dd.concat([df_count, df_installment], axis=1)
df_final = dd.concat([df_mean, df_count_inst], axis=1)

df_final = df_final.reset_index()
df_final=df_final.sort_values(by='SK_ID_CURR')

df_final.round(2).head()

,SK_ID_PREV,SK_ID_CURR,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,INSTALMENTS_paid,SK_DPD,SK_DPD_DEF,STATUS_Active,STATUS_Completed,STATUS_Demand,STATUS_Initialized,MONTHS_BALANCE_count,MONTHS_BALANCE_latest,INSTALMENTS_paid_latest,NAME_CONTRACT_STATUS_latest,NAME_CONTRACT_STATUS_red_latest
4906,1489396,100006.0,0.00,270000.00,NaN,0.00,NaN,NaN,0.00,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,1.00,0.00,0.0,0.0,6,-1,0.0,Active,Active
8001,1843384,100011.0,54482.11,164189.19,2432.43,2432.43,0.0,0.0,3956.22,4843.06,4520.07,52402.09,54433.18,54433.18,0.05,0.05,0.0,0.0,25.77,0.00,0.00,1.00,0.00,0.0,0.0,74,-2,33.0,Active,Active
9770,2038692,100013.0,18159.92,131718.75,6350.00,5953.12,0.0,0.0,1454.54,7168.35,6817.17,17255.56,18101.08,18101.08,0.26,0.24,0.0,0.0,18.72,0.01,0.01,1.00,0.00,0.0,0.0,96,-1,22.0,Active,Active
14661,2594025,100021.0,0.00,675000.00,NaN,0.00,NaN,NaN,0.00,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,0.41,0.59,0.0,0.0,17,-2,0.0,Completed,Completed
5029,1499902,100023.0,0.00,135000.00,NaN,0.00,NaN,NaN,0.00,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,1.00,0.00,0.0,0.0,8,-4,0.0,Active,Active


In [ ]:
size_df_final = [df_final.shape[0].compute(),  df_final.shape[1]]

print('The condensed dataset has', size_df_final[0], 'rows and', size_df_final[1], 'features.')
print('Initial size was', size_df[0], 'rows and', size_df[1], 'features.')

The condensed dataset has 104307 rows and 30 features.
Initial size was 3840312 rows and 23 features.


In [ ]:
nan_overview_df = dfunc.nan_type_overview_dd(df_final, size_df_final[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

,type,NaN[abs],NaN[%]
SK_ID_PREV,int64,0,0.000000
SK_ID_CURR,float64,0,0.000000
AMT_BALANCE,float64,0,0.000000
AMT_CREDIT_LIMIT_ACTUAL,float64,0,0.000000
AMT_DRAWINGS_ATM_CURRENT,float64,31794,30.500000
AMT_DRAWINGS_CURRENT,float64,0,0.000000
AMT_DRAWINGS_OTHER_CURRENT,float64,31794,30.500000
AMT_DRAWINGS_POS_CURRENT,float64,31794,30.500000
AMT_INST_MIN_REGULARITY,float64,0,0.000000
AMT_PAYMENT_CURRENT,float64,31862,30.500000


This modified data set can now be merged with the 'previous applictions' dataset.

# Export

In [ ]:
%%capture
! mkdir home_folder+'cleaned/'
df_final.to_csv(home_folder+'cleaned/HC_credit_card_cleaned.csv',
                 index=False, single_file = True)